# Pretrain Dataset
* Saves training data to disk

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb")
@nbinclude("Data.ipynb");

In [ ]:
name = "all/Transformer/v1"
set_logging_outdir(name);

In [ ]:
import HDF5
import JSON
import MLUtils
import Random
import StatsBase: mean, sample

# Featurization

In [ ]:
featurize(sentence::Vector{wordtype}, config, rng) = featurize(;
    sentence = sentence,
    max_seq_len = config[:max_sequence_length],
    vocab_sizes = config[:base_vocab_sizes],
    pad_tokens = config[:pad_tokens],
    cls_tokens = config[:cls_tokens],
    mask_tokens = config[:mask_tokens],
    rng = rng,
);

In [ ]:
function featurize(;
    sentence::Vector{wordtype},
    max_seq_len,
    vocab_sizes,
    pad_tokens,
    cls_tokens,
    mask_tokens,
    rng,
)
    sentence = subset_sentence(sentence, max_seq_len; recent = false, rng = rng)
    tokens = get_token_ids(sentence, max_seq_len, pad_tokens, cls_tokens)

    positions = Dict(
        x => Dict(y => zeros(Int32, max_seq_len) for y in ALL_METRICS) for x in ALL_MEDIUMS
    )
    weights = Dict(
        x => Dict(y => zeros(Float32, max_seq_len) for y in ALL_METRICS) for
        x in ALL_MEDIUMS
    )
    labels = Dict(
        x => Dict(y => zeros(Float32, max_seq_len) for y in ALL_METRICS) for
        x in ALL_MEDIUMS
    )
    userids = Dict(
        x => Dict(y => zeros(Int32, max_seq_len) for y in ALL_METRICS) for x in ALL_MEDIUMS
    )
    for i::Int32 = 1:max_seq_len
        is_manga = (extract(tokens, :itemid)[i] < num_items("manga"))
        is_anime =
            num_items("manga") <=
            extract(tokens, :itemid)[i] <
            num_items("manga") + num_items("anime")
        has_rating =
            0 < extract(tokens, :rating)[i] <= vocab_sizes[get_wordtype_index(:rating)]
        has_watch =
            get_status(:plan_to_watch) <
            extract(tokens, :status)[i] <=
            vocab_sizes[get_wordtype_index(:status)]
        has_plantowatch = extract(tokens, :status)[i] == get_status(:plan_to_watch)
        has_drop =
            get_status(:none) <
            extract(tokens, :status)[i] <=
            vocab_sizes[get_wordtype_index(:status)]
        if !(has_rating || has_watch || has_plantowatch || has_drop)
            continue
        end
        @assert xor(is_manga, is_anime)
        if is_manga
            medium = "manga"
            mediaid = extract(tokens, :itemid)[i]
        elseif is_anime
            medium = "anime"
            mediaid = extract(tokens, :itemid)[i] - num_items("manga")
        else
            @assert false
        end

        # mask and make predictions for 15% of tokens
        if rand(rng) > 0.15
            continue
        end
        if has_rating
            positions[medium]["rating"][i] = mediaid
            labels[medium]["rating"][i] = extract(tokens, :rating)[i]
            weights[medium]["rating"][i] = 1
            userids[medium]["rating"][i] = extract(tokens, :userid)[i]
        end
        if has_watch
            positions[medium]["watch"][i] = mediaid
            labels[medium]["watch"][i] = 1
            weights[medium]["watch"][i] = 1
            userids[medium]["watch"][i] = extract(tokens, :userid)[i]
        end
        if has_plantowatch
            positions[medium]["plantowatch"][i] = mediaid
            labels[medium]["plantowatch"][i] = 1
            weights[medium]["plantowatch"][i] = 1
            userids[medium]["plantowatch"][i] = extract(tokens, :userid)[i]
        end
        if has_drop
            positions[medium]["drop"][i] = mediaid
            labels[medium]["drop"][i] = extract(tokens, :status)[i] <= get_status(:dropped)
            weights[medium]["drop"][i] = 1
            userids[medium]["drop"][i] = extract(tokens, :userid)[i]
        end

        # do bert masking        
        r = rand(rng)
        if r <= 0.8
            for j = 1:length(tokens)
                if j ∉ get_wordtype_index.([:userid, :position])
                    tokens[j][i] = mask_tokens[j]
                end
            end
        elseif r <= 0.9
            for j = 1:length(tokens)
                if j ∉ get_wordtype_index.([:userid, :position])
                    if eltype(vocab_sizes[j]) == Int32
                        tokens[j][i] = rand(rng, 0:vocab_sizes[j]-1)
                    elseif eltype(tokens[j]) == Float32
                        tokens[j][i] = rand(rng) * vocab_sizes[j]
                    else
                        @assert false
                    end
                end
            end
        else
            nothing
        end
    end

    tokens, positions, labels, weights
end;

# Data collection

In [ ]:
function shuffle_training_data(rng, sentences, max_sequence_length, max_document_length)
    order = Random.shuffle(rng, 1:length(sentences))
    S = eltype(sentences)
    W = eltype(sentences[1])

    # patition tokens into minibatches
    batched_sentences = Vector{S}()
    sentence = Vector{W}()
    for i in order
        subset =
            subset_sentence(sentences[i], max_document_length; recent = false, rng = rng)
        for token in subset
            push!(sentence, token)
            if length(sentence) == max_sequence_length
                push!(batched_sentences, sentence)
                sentence = Vector{W}()
            end
        end
    end
    if length(sentence) > 0
        push!(batched_sentences, sentence)
    end
    batched_sentences
end;

In [ ]:
function get_sentences(rng, cls_tokens)
    sentences = collect(values(get_training_data(cls_tokens)))
    Random.shuffle!(rng, sentences)
    cutoff = Int(round(0.99 * length(sentences))) # TODO switch to 0.999 for prod
    training = sentences[1:cutoff]
    validation = sentences[cutoff+1:end]
    training, validation
end;

# Configuration

In [ ]:
function set_rngs(seed)
    rng = Random.Xoshiro(seed)
    Random.seed!(rand(rng, UInt64))
    rng
end;

In [ ]:
function create_training_config()
    # itemid, rating, updated_at, status, position, userid
    max_sequence_length = 1024
    base_vocab_sizes = (
        Int32(sum(num_items.(ALL_MEDIUMS)) - 1),
        Float32(10),
        Float32(1),
        Int32(get_status(:rewatching)),
        Int32(max_sequence_length - 1),
        Int32(num_users() - 1),
    )
    d = (
        # tokenization
        base_vocab_sizes = base_vocab_sizes,
        cls_tokens = base_vocab_sizes .+ Int32(1),
        pad_tokens = base_vocab_sizes .+ Int32(2),
        mask_tokens = base_vocab_sizes .+ Int32(3),
        vocab_sizes = base_vocab_sizes .+ Int32(4),
        vocab_types = ("int", "float", "float", "int", "int", "none"),
        media_sizes = Dict(m => num_items(m) for m in ALL_MEDIUMS),
        # data
        max_document_length = Inf, # TODO experiment with subsampling
        chunk_size = 2^14,
        training_epoch_size = -1,
        validation_epoch_size = -1,
        # model
        max_sequence_length = max_sequence_length,
        mode = "pretrain",
    )
    @assert d[:max_document_length] >= d[:max_sequence_length]
    @assert length(d[:vocab_sizes]) == length(d[:vocab_types])
    d
end;

In [ ]:
function set_epoch_size(config, training_sentences, validation_sentences)
    for (s, split) in
        zip([training_sentences, validation_sentences], ["training", "validation"])
        num_tokens = Int64(sum(min.(length.(s), config[:max_document_length])))
        @info "Number of $split tokens: $(num_tokens)"
        @info "Number of $split sentences: $(length(s))"
        config = @set config[Symbol("$(split)_epoch_size")] =
            div(num_tokens, config[:max_sequence_length], RoundUp)
    end
    config
end;

In [ ]:
function setup_training(config, outdir)
    mkpath(outdir)
    fn = joinpath(outdir, "config.json")
    open(fn, "w") do f
        write(f, JSON.json(config))
    end
    for split in ["training", "validation"]
        fn = joinpath(outdir, split)
        mkpath(fn)
        for x in readdir(fn, join = true)
            rm(x, recursive = true)
        end
    end
end;

# Disk I/O

In [ ]:
function save_features(sentences, config, rng, filename)
    sentences = shuffle_training_data(
        rng,
        sentences,
        config[:max_sequence_length],
        config[:max_document_length],
    )
    features = []
    for x in sentences
        push!(features, featurize(x, config, rng))
    end

    d = Dict{String,AbstractArray}()
    collate = MLUtils.batch
    embed_names = ["itemid", "rating", "updated_at", "status", "position", "userid"]
    for (i, name) in Iterators.enumerate(embed_names)
        d[name] = collate([x[1][i] for x in features])
    end
    for medium in ALL_MEDIUMS
        for metric in ALL_METRICS
            stem = "$(medium)_$(metric)"
            d["positions_$stem"] = collate([x[2][medium][metric] for x in features])
            d["labels_$stem"] = collate([x[3][medium][metric] for x in features])
            d["weights_$stem"] = collate([x[4][medium][metric] for x in features])
        end
    end
    HDF5.h5open(filename, "w") do f
        for (k, v) in d
            f[k, deflate = 6] = v
        end
    end
end;

In [ ]:
function save_epoch(sentences, config, epoch, outdir)
    workers = 16
    outdir = joinpath(outdir, "$epoch")
    mkpath(outdir)
    rng = Random.Xoshiro(epoch)
    Random.shuffle!(rng, sentences)
    batches = collect(Iterators.partition(sentences, config[:chunk_size]))
    seeds = [rand(rng, UInt64) for _ = 1:length(batches)]
    @showprogress for idxs in Iterators.partition(1:length(batches), workers)
        @sync for i in idxs
            Threads.@spawn save_features(
                batches[i],
                config,
                Random.Xoshiro(seeds[i]),
                joinpath(outdir, "$(i-1).h5"),
            )
        end
        GC.gc()
    end
end;

In [ ]:
function save_epochs(num_epochs)
    rng = set_rngs(20221221)
    config = create_training_config()
    @info "loading data"
    training_sentences, validation_sentences = get_sentences(rng, config[:cls_tokens])
    config = set_epoch_size(config, training_sentences, validation_sentences)
    outdir = get_data_path(joinpath("alphas", name))
    setup_training(config, outdir)
    for epoch = 0:num_epochs-1
        @time save_epoch(
            validation_sentences,
            config,
            epoch,
            joinpath(outdir, "validation"),
        )
        @time save_epoch(training_sentences, config, epoch, joinpath(outdir, "training"))
    end
end;

In [ ]:
save_epochs(64)